In [ ]:
import zipfile
import os
from pathlib import Path
from datetime import datetime
import pandas as pd
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import numpy as np

# Unzip the dropped funds data

In [ ]:
def unzip_files(PATH): # Function requires path to zip folder as a argument
    with zipfile.ZipFile(PATH, 'r') as zip_file:
        # Function will create Unzipped folder within working directory and extract all files in Unzipped folder
        zip_file.extractall('Unzipped') 

In [ ]:
# Assuming zip folder is in same directory as of this notebook, if not then need to pass correct path
unzip_files("HF Archives.zip") 

# Process the unzipped folder and load latest Data

In [ ]:
# recursively iterate throguth the unzipped folder and record all timestamps and its parent folder path
pathlist = Path("Unzipped").glob('**/*')
Times= []
for path in pathlist:
     if os.path.splitext(path)[1] == '.txt':
         with open(path , 'r') as f:
             time =  f.read()
             Times.append((str(path.parents[0]) , datetime.strptime(time, "%Y%m%d")))

# return latest timestamps and latest funds directory path
funds_timstamp = [x[1] for x in Times]
latest_timestamp = max(funds_timstamp)
latest_timestamp_index = funds_timstamp.index(latest_timestamp)
latest_funds_parent_path = Times[latest_timestamp_index][0]

# Loop through all the csv files in latest funds directory and load the data
dataframs = []
dataframe_names= []
latest_fund_data = Path(latest_funds_parent_path).glob('**/*.csv')
for data_path in latest_fund_data:
    dataframe_names.append(data_path.stem)
    temp_df = pd.read_csv(data_path)
    dataframs.append(temp_df)
    
# Create Seperate Dataframs and display confirmation message with loaded dataframs name   
for i in range(len(dataframs)):
    globals()[f"DF_{dataframe_names[i]}"] = dataframs[i]
    print(f"Letest DF_{dataframe_names[i]} data has been loaded! It has {dataframs[i].shape[0]} rows and {dataframs[i].shape[1]} columns")

In [ ]:
# convert Date column from object to datetime datatype 
DF_HF_ROR['Date'] = pd.to_datetime(DF_HF_ROR['Date'])
DF_HF_ASSETS['Date']= pd.to_datetime(DF_HF_ASSETS['Date'])


# Data Analysis


### 1) What are the top and bottom ITD performers:
       - By Cumulative ROR?
       - By Annualized ROR (Minimum 12 months of data)?

#### By Cumulative ROR

In [ ]:
# Sort data by Fund_ID and Date
filtered_df = DF_HF_ROR.sort_values(by=['Fund_ID','Date'])
# calculate the cumulative ROR
filtered_df['Interest_factor'] = (filtered_df['Performance']/100 +1)
filtered_df['Cumulative_ret'] = filtered_df.groupby(['Fund_ID']).Interest_factor.cumprod()-1

In [ ]:
# return ITB Cumulative ROR for each fund  by getting latest row in each fund and sort in decending order
Cumulative_ret_df = filtered_df.loc[filtered_df.groupby('Fund_ID').Date.idxmax(), ['Fund_ID','Date', 'Cumulative_ret']]
Cumulative_ret_df.sort_values(by=['Cumulative_ret'], ascending=False, inplace=True)

In [ ]:
Cumulative_ret_df

In [ ]:
#print The result to console
Top_ITD_performer_by_cumulative_ROR = Cumulative_ret_df.head(1)
print(f'Top ITD Performer by Cumulative ROR is:\n')
print(f"Fund ID : {Top_ITD_performer_by_cumulative_ROR['Fund_ID'].values}")
print(f"Cumulative ROR : {'{:.2%}'.format(Top_ITD_performer_by_cumulative_ROR['Cumulative_ret'].values[0])}\n")
Bottom_ITD_performer_by_cumulative_ROR = Cumulative_ret_df.tail(1)
print(f'Bottom ITD Performer by Cumulative ROR is:\n')
print(f"Fund ID : {Bottom_ITD_performer_by_cumulative_ROR['Fund_ID'].values}")
print(f"Cumulative ROR : {'{:.2%}'.format(Bottom_ITD_performer_by_cumulative_ROR['Cumulative_ret'].values[0])}\n")

In [ ]:
filtered_df.loc[filtered_df['Fund_ID']== Top_ITD_performer_by_cumulative_ROR['Fund_ID'].values[0], ['Date','Cumulative_ret']].plot( x='Date', y='Cumulative_ret', title = 'Top perfomer fund\'s cumulative return')

In [ ]:
filtered_df.loc[filtered_df['Fund_ID']== Bottom_ITD_performer_by_cumulative_ROR['Fund_ID'].values[0], ['Date','Cumulative_ret']].plot( x='Date', y='Cumulative_ret', title = 'Bottom perfomer fund\'s cumulative return')

#### By Annualized ROR (Minimum 12 months of Data)

In [ ]:
# Filter the data with fund that has minimum 12 months of data
df =DF_HF_ROR['Fund_ID'].value_counts()
funds_with_minimum_36_months = df[df >= 12].index
f_df = DF_HF_ROR[DF_HF_ROR['Fund_ID'].isin(funds_with_minimum_36_months)]
f_df.shape

In [ ]:
# Sort data by Fund_ID and Date
f_df = f_df.sort_values(by=['Fund_ID','Date'])
# calculate the cumulative ROR
f_df['Interest_factor'] = (f_df['Performance']/100 +1)
f_df['Cumulative_ret (ITD)'] = f_df.groupby(['Fund_ID']).Interest_factor.cumprod() -1

In [ ]:
# create dataframe with ITD cumulative ror for each funds 
Cumulative_ret_df = f_df.loc[f_df.groupby('Fund_ID').Date.idxmax(), ['Fund_ID','Date', 'Cumulative_ret (ITD)']]
Cumulative_ret_df.sort_values(by=['Fund_ID'], inplace=True)

In [ ]:
# Set Fund_ID as a index for cumulative df, it will be easy to slice and concat dataframe
Cumulative_ret_df.set_index('Fund_ID' , inplace = True)

In [ ]:
# create datafram with number of periods of each fund and set fund_id as index
number_of_periods = f_df.groupby('Fund_ID').agg({'Fund_ID': 'size'}).rename(columns={'Fund_ID': 'Number_of_Periods'})


In [ ]:
# create Annualized dataframe
Annualized_df = pd.concat([Cumulative_ret_df,number_of_periods] , axis =1)

In [ ]:
# create ITD annualized ROR for funds (Minimum 12 months data)
Annualized_df['Annualized ROR (ITD)'] = ((1+ Annualized_df['Cumulative_ret (ITD)'])**(12/Annualized_df['Number_of_Periods']))-1

In [ ]:
#print The result to console
Top_ITD_performer_by_Annualized_ROR = Annualized_df.sort_values('Annualized ROR (ITD)', ascending=False).head(1)
print(f'Top ITD Performer by Annualized ROR is:\n')
print(f"Fund ID : {Top_ITD_performer_by_Annualized_ROR.index.values}")
print(f"Annualized ROR : {'{:.2%}'.format(Top_ITD_performer_by_Annualized_ROR['Annualized ROR (ITD)'].values[0])}\n")
Bottom_ITD_performer_by_Annualized_ROR = Annualized_df.sort_values('Annualized ROR (ITD)', ascending=False).tail(1)
print(f'Bottom ITD Performer by Annualized ROR is:\n')
print(f"Fund ID : {Bottom_ITD_performer_by_Annualized_ROR.index.values}")
print(f"Annualized ROR : {'{:.2%}'.format(Bottom_ITD_performer_by_Annualized_ROR['Annualized ROR (ITD)'].values[0])}\n")

### What are the top and bottom 10 YTD performers by ROR of 2018?

In [ ]:
# filtered dataframe for year 2018
filtered_df = DF_HF_ROR[DF_HF_ROR['Date'].dt.year == 2018].copy()

In [ ]:
filtered_df['Interest_factor'] = (filtered_df['Performance']/100 + 1)

In [ ]:
# Note = With the assumption that performace given is Discrete Return
top10_2018_ytd_funds = filtered_df.groupby(["Fund_ID"]).Interest_factor.prod().nlargest(10).reset_index()

In [ ]:
# Note = With the assumption that performace given is Discrete Return
Bottom10_2018_ytd_funds = filtered_df.groupby(["Fund_ID"]).Interest_factor.prod().nsmallest(10).reset_index()

In [ ]:
# Add YTD ROR percentage column
top10_2018_ytd_funds['YTD_ROR'] = (top10_2018_ytd_funds['Interest_factor'] -1 )*100
Bottom10_2018_ytd_funds['YTD_ROR'] = (Bottom10_2018_ytd_funds['Interest_factor'] -1 )*100

In [ ]:
# Top 10 Performer for 2018 by YTD ROR
x_labels = top10_2018_ytd_funds['Fund_ID'].values
YTD_Return = top10_2018_ytd_funds['YTD_ROR']
# Plot the figure.
plt.figure(figsize=(10, 7))
ax = YTD_Return.plot(kind="bar")
ax.set_title("Top 10 YTD ROR for 2018")
ax.set_xlabel("Fund ID")
ax.set_ylabel("YTD ROR (%)")
ax.set_xticklabels(x_labels)
rects = ax.patches

# Make PCT value as labels.
labels = pd.Series(["{0:.2f}%".format(val) for val in top10_2018_ytd_funds['YTD_ROR']], index = top10_2018_ytd_funds.index)

# add lables to graph
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2, height +0.5 , label, ha="center", va="bottom"
    )

plt.show()

In [ ]:
# Bottom 10 Performer for 2018 by YTD ROR
x_labels = Bottom10_2018_ytd_funds['Fund_ID'].values
YTD_Return = Bottom10_2018_ytd_funds['YTD_ROR']
# Plot the figure.
plt.figure(figsize=(10, 6))
ax = YTD_Return.plot(kind="bar")
ax.set_title("Bottom 10 YTD ROR for 2018")
ax.set_xlabel("Fund ID")
ax.set_ylabel("YTD ROR (%)")
ax.set_xticklabels(x_labels)

rects = ax.patches

# Make PCT value as labels
labels = pd.Series(["{0:.2f}%".format(val) for val in Bottom10_2018_ytd_funds['YTD_ROR']], index = Bottom10_2018_ytd_funds.index)


# add lables to graph
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2, height - 3, label, ha="center", va="bottom"
    )

plt.show()


### Assuming an annualized risk free return of 2%, which funds show the best risk adjusted returns (minimum 36 months of data)?

In [ ]:
# Filter the data with fund that has minimum 36 months of data
df =DF_HF_ROR['Fund_ID'].value_counts()
funds_with_minimum_36_months = df[df >= 36].index
f_df = DF_HF_ROR[DF_HF_ROR['Fund_ID'].isin(funds_with_minimum_36_months)]

In [ ]:
# Find the annualized return of funds (Minimum 36 months of data)

# Sort data by Fund_ID and Date
f_df = f_df.sort_values(by=['Fund_ID','Date'])
# calculate the cumulative ROR
f_df['Interest_factor'] = (f_df['Performance']/100 +1)
f_df['Cumulative_ret (ITD)'] = f_df.groupby(['Fund_ID']).Interest_factor.cumprod()-1
# create dataframe with ITD cumulative ror for each funds 
Cumulative_ret_df = f_df.loc[f_df.groupby('Fund_ID').Date.idxmax(), ['Fund_ID','Date', 'Cumulative_ret (ITD)']]
Cumulative_ret_df.sort_values(by=['Fund_ID'], inplace=True)
# Set Fund_ID as a index for cumulative df, it will be easy to slice and concat datafram
Cumulative_ret_df.set_index('Fund_ID' , inplace = True)
# create datafram with number of periods of each fund and set fund_id as index
number_of_periods = f_df.groupby('Fund_ID').agg({'Fund_ID': 'size'}).rename(columns={'Fund_ID': 'Number_of_Periods'})
# create Annualized dataframe
Annualized_df = pd.concat([Cumulative_ret_df,number_of_periods] , axis =1)
# create ITD annualized ROR for funds (Minimum 12 months data)
Annualized_df['Annualized ROR (ITD)'] = ((1+ Annualized_df['Cumulative_ret (ITD)'])**(12/Annualized_df['Number_of_Periods']))-1

In [ ]:
# Find the annualized Standrd Deviation of funds (Minimum 36 months of data)

Annualized_STD = f_df.groupby('Fund_ID').agg({'Interest_factor': 'std'}).rename(columns={'Interest_factor': 'Standard deviation'})
Annualized_STD['Annualized Standard deviation'] = Annualized_STD['Standard deviation'] * np.sqrt(12)

In [ ]:
f_df[f_df['Fund_ID']== 185684]

In [ ]:
# Annualized risk free return rate is 2%
Annualized_Risk_Free_ROR = 2/100

In [ ]:
# calculate Annualized risk adjusted return
Annualized_df = pd.concat([Annualized_df,Annualized_STD] , axis =1 )
Annualized_df['Risk_Adjusted_return(Sharp Ratio)'] = (Annualized_df['Annualized ROR (ITD)'] - Annualized_Risk_Free_ROR)/Annualized_df['Annualized Standard deviation']

In [ ]:
# Sort the datafram by Risk_Adjusted_return(Sharp Ratio) in descending order
Annualized_df.sort_values('Risk_Adjusted_return(Sharp Ratio)',ascending=False,inplace = True)

In [ ]:
# Best risk adjusted return fund
Best_Fund_By_RAR = Annualized_df.head(1)
print(f'Best Fund by Annualized Risk Adjusted Return (Sharp Ratio) is:\n')
print(f"Fund ID : {Best_Fund_By_RAR.index.values}")
print(f"Annualized Risk Adjusted Return (Sharp Ratio) : {Best_Fund_By_RAR['Risk_Adjusted_return(Sharp Ratio)'].values[0]}")
print(f"Annualized Standard Deviation (volatility) : {Best_Fund_By_RAR['Annualized Standard deviation'].values[0]}")
print(f"Annualized ROR : {'{:.2%}'.format(Best_Fund_By_RAR['Annualized ROR (ITD)'].values[0])}")
print(f"Number of Months : {Best_Fund_By_RAR['Number_of_Periods'].values[0]}")
print(f"Cumulative ROR : {'{:.2%}'.format(Best_Fund_By_RAR['Cumulative_ret (ITD)'].values[0])}\n")

#### Create a composite index of the best performing 10 funds over 2018 and track the performance over the year.
- Assume the portfolio is reweighted at the beginning of each month based on the prior month’s top performers
- Assume equal weighting each month (each fund is 10% of the portfolio)
- What would be the expected rate of return at the end of 2018?
- Which fund contributed the most to the composite’s total rate of return? What its contribution to the total contribution of the pro-forma index?

In [ ]:
# filter the data
filtered_df = DF_HF_ROR[(DF_HF_ROR['Date'] >= '12/31/2017') & (DF_HF_ROR['Date'] <= '12/31/2018')].copy()
filtered_df = filtered_df.sort_values(['Fund_ID', 'Date'])

In [ ]:
# Return Previous months performace 
filtered_df['Previous_month_performance'] = filtered_df.groupby(['Fund_ID']).Performance.shift(1)

# Drop null values from previous month performance column: 
#This will drop all the rows for December 2017 and only keep rows for 2018
filtered_df = filtered_df[filtered_df['Previous_month_performance'].notna()]

In [ ]:
# Create Composite index
# -- This will return top 10 best performing funds based on previous month performance for each month
# -- Using MultipleIndex to store Fund_ID, Date, and current month performance 
Composite_Index = filtered_df.set_index(['Fund_ID' , 'Date', 'Performance']).groupby('Date').Previous_month_performance.nlargest(10)
Composite_Index = Composite_Index.reset_index(level=[0,1,1])
Composite_Index['Performance'] = Composite_Index.index
Composite_Index['Performance_decimal'] = Composite_Index['Performance']/100

Composite_Index.reset_index(drop = True , inplace = True)

In [ ]:
# Add Weights column : in our case we have equal weight of 10%
Composite_Index['Weight_PCT'] = 0.10

# Calculate the weighted return
Composite_Index['Weighted_Performance_decimal'] = (Composite_Index['Performance_decimal']) * Composite_Index['Weight_PCT']
Composite_Index

In [ ]:
# Return Monthly performance of composite index based on weighted return of the funds
Composite_Index_monthly_perfomance = Composite_Index.groupby('Date').Weighted_Performance_decimal.sum().reset_index()

# Expected Rate of return at the end of 2018 for composite index
Expected_ROR_2018 = (Composite_Index_monthly_perfomance.sum()/12)
print(f"Expected (Average) ROR at the end of 2018 for the composite fund is: {'{:.2%}'.format(Expected_ROR_2018.values[0])}")

In [ ]:
# Composite Index Monthly Performance
Composite_Index_monthly_perfomance.plot.line(x = 'Date' , y ='Weighted_Performance_decimal', title = 'Composite Index Monthly Performance')



In [ ]:
Composite_Index_monthly_perfomance

In [ ]:
# Total ROR at the end of 2018
Composite_Index_monthly_perfomance['Interest_factor'] = Composite_Index_monthly_perfomance['Weighted_Performance_decimal']+1
Total_ROR_at_the_end_2018 = (Composite_Index_monthly_perfomance.Interest_factor.prod()-1)*100
print (f'Total ROR at the end of 2018 for the composite index is : {"{0:.2f}%".format(Total_ROR_at_the_end_2018)}')

#fund contributed the most to the composite’s total rate of return

Composite_Index['Interest_factor'] = Composite_Index['Weighted_Performance_decimal'] + 1

Fund_Contribution  = Composite_Index.groupby('Fund_ID').Interest_factor.prod()
Fund_Contribution = Fund_Contribution.reset_index()
Fund_Contribution = Fund_Contribution.sort_values('Interest_factor', ascending = False)
Fund_Contribution.reset_index(drop = True, inplace =True)
Fund_Contribution

Fund_Contributted_the_most = Fund_Contribution.head(1)
print(f'The fund contributed the most to Composite total ROR is:')
print(f"Fund ID : {Fund_Contributted_the_most['Fund_ID'].values[0]}")

# (Fund_Contribution['Interest_factor'].div(Total_ROR_at_the_en//////////d_2018)


#### Provide the Top/Bottom 10 increases in AUM since the beginning of 2018 

In [ ]:
# filteretd the dataframe to get data since beging of 2018
# Assets provided millions.

filtered_df = DF_HF_ASSETS[DF_HF_ASSETS['Date'] >= '2017-12-31'].copy()

In [ ]:
#sort the datafram in ascending order
filtered_df.sort_values('Date' , ascending = True, inplace = True)

# Calculate AMU increses since the beging of 2018
Grouped_data=filtered_df.groupby(by='Fund_ID').agg(['first','last'])
Grouped_data.columns =  ['Starting Date (End of 2017)', 'Latest Date', 'AUM at start of 2018 (Millions)', 'Latest AUM (Millions)']
Grouped_data['AUM Increase (Millions)'] =  Grouped_data['Latest AUM (Millions)'] - Grouped_data['AUM at start of 2018 (Millions)']

# Order the datafram by AUM Increases in Descending order and return top and bottom 10 funds
Grouped_data.sort_values('AUM Increase (Millions)', ascending = False, inplace = True )
Top10_AUM_Increased_since_2018 = Grouped_data.head(10)
Bottom10_AUM_Increased_since_2018 = Grouped_data.tail(10)

In [ ]:
# Top 10 increase since the beging of 2018
Top10_AUM_Increased_since_2018

In [ ]:
x_labels = Top10_AUM_Increased_since_2018.index.values
YTD_Return = Top10_AUM_Increased_since_2018['AUM Increase (Millions)']
# Plot the figure.
plt.figure(figsize=(12, 8))
ax = YTD_Return.plot(kind="bar")
ax.set_title("Top 10 AUM increases since begning of 2018")
ax.set_xlabel("Fund ID")
ax.set_ylabel("AUM increase in Millions")
ax.set_xticklabels(x_labels)
rects = ax.patches

# Make PCT value as labels.
labels = pd.Series(["{0:.2f} M".format(val) for val in Top10_AUM_Increased_since_2018['AUM Increase (Millions)']], index = Top10_AUM_Increased_since_2018.index)

# add lables to graph
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2, height + 1.5 , label, ha="center", va="bottom"
    )

plt.show()

In [ ]:
# Bottom 10 increase since the beging of 2018
Bottom10_AUM_Increased_since_2018

In [ ]:
x_labels = Bottom10_AUM_Increased_since_2018.index.values
YTD_Return = Bottom10_AUM_Increased_since_2018['AUM Increase (Millions)']
plt.figure(figsize=(12, 8))
ax = YTD_Return.plot(kind="bar")
ax.set_title("Bottom 10 AUM increases since begning of 2018")
ax.set_xlabel("Fund ID")
ax.set_ylabel("AUM increase in Millions")
ax.set_xticklabels(x_labels)

rects = ax.patches

# Make PCT value as labels
labels = pd.Series(["{0:.2f}M".format(val) for val in Bottom10_AUM_Increased_since_2018['AUM Increase (Millions)']], index = Bottom10_AUM_Increased_since_2018.index)


# add lables to graph
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width() / 2, height - 2500, label,ha="center", va="bottom"
    )

plt.show()

##### Why AUM Tracking is Important ?

##### AUM refers to the total market value of assets being managed fund manager or institution. the exact definition of AUM varies by institution; some include bank deposits, mutual funds, and cash in their computation, while others only consider the funds that investors have given an advisor to trade on their behalf.

##### AUM reflects management performance and size. Oftenly, a fund's management fees and expenses are calculated as a percentage of AUM. So, it is imperative to track AUM increases. Many firm will monitor AUM as it relates to determining the strength of the company. Investment companies also use AUM as a marketing tool to attract new investors. AUM can help investors get an indication of the size of a company's operations relative to its competitors. 

#### Segment managers into appropriate buckets by AUM size and evaluate performance (ROR) across the segments. What results and general trends do you see?

In [ ]:
# return latest AUM of the Funds
latest_fund_AUM = DF_HF_ASSETS.sort_values(['Fund_ID','Date']).groupby('Fund_ID').last()

In [ ]:
# Order Funds by AUM by descending order
latest_fund_AUM.sort_values('Assets', ascending = False, inplace = True)

In [ ]:
# Segment Funds based on AUM
bins = [0,100,200,300,400,500,600,700,800,900,1000,90000]
labels = ["AUM Under 100 Millions","AUM Between 100 to 200 Millions","AUM Between 200 to 300 Millions","AUM Between 300 to 400 Millions","AUM Between 400 to 500 Millions","AUM Between 500 to 600 Millions","AUM Between 600 to 700 Millions", "AUM Between 700 to 800 Millions", "AUM Between 800 to 900 Millions","AUM Between 900 to 1000 Millions","AUM Over 1000 Millions"]

latest_fund_AUM['Segments'] = pd.cut(latest_fund_AUM['Assets'],bins=bins, labels=labels , include_lowest=True)

##### AUM can change beacuse of two reasons 1) Price changes 2) New investments
##### I could have calculated performance from AUM but to do so i need to assume that chnages in AUM is only beacase of price chnage and there is no new investments, which is not realistic assumption and there is no explicit instruction provided so decided to use ROR data (Cumulative ROR of the funds) for performance evaluation.  
##### AUM dataframe has less funds then ROR dataframe so using left outer join (Left table is AUM Data) to return all the funds from AUM dataframe and only the matched funds from cumulative dataframe

In [ ]:
# Calculate the cumulative ROR of Funds 
f_df = DF_HF_ROR.copy()
# Sort data by Fund_ID and Date
f_df = f_df.sort_values(by=['Fund_ID','Date'])
# calculate the cumulative ROR
f_df['Interest_factor'] = (f_df['Performance']/100 +1)
f_df['Cumulative_ret (ITD)'] = f_df.groupby(['Fund_ID']).Interest_factor.cumprod()-1
# create dataframe with ITD cumulative ROR for each funds 
Cumulative_ret_df = f_df.loc[f_df.groupby('Fund_ID').Date.idxmax(), ['Fund_ID','Date', 'Cumulative_ret (ITD)']]
Cumulative_ret_df.sort_values(by=['Fund_ID'], inplace=True)
# Set Fund_ID as a index for cumulative df, it will be easy to slice and concat datafram
Cumulative_ret_df.set_index('Fund_ID' , inplace = True)

In [ ]:
# update column to indetify data is coming from ROR dataframe
Cumulative_ret_df.columns =['ROR_Date', 'Cumulative_ROR (ITD)']

In [ ]:
# create combine datafram of AUM dataframe and cumulative ROR dataframe to compare perfomance across segements
Combine_Df = latest_fund_AUM.join(Cumulative_ret_df , how="left")

##### Segment Analysis
 - Segment Size Distribution
 - Performance Accrose Segemnts

In [ ]:
# Segment Size Distribution 
Combine_Df['Segments'].value_counts(normalize =True).plot( kind = 'bar')
Combine_Df['Segments'].value_counts(normalize = True)


In [ ]:
Segments_average_cumulative_ROR = Combine_Df.groupby('Segments')['Cumulative_ROR (ITD)'].mean()
print("Segment's average cumulative ROR : ")
print(Segments_average_cumulative_ROR)
print("=================================================")
Segments_volatility = Combine_Df.groupby('Segments')['Cumulative_ROR (ITD)'].std()
print("Segment's volatility: Standard Deviation")
print(Segments_volatility)

#### Segment Observation:

##### Distribution:
  * Segment size is are very right skewd as majority of funds has AUM under 100 millions (71% of funds)

##### Performance across segments:
  * AUM between 800 to 900 millions segment has lowest volatility which suggests that their fund managers tend to take less risk. This Segment also has second lowest average cumulative ROR.
  
  * AUM between 900 to 1000 millions and AUM over 1000 Millions has highest and second highest average cumulative ROR but their fund managers has taken highest and second highest risk respectively.
  
  * AUM Under 100 Millions has the lowest average cumulative ROR and their fund manager has taken medium risk.
  
  * Based on above mentioned observations, we can say that Higher AUM funds managers has taken higher risk and scored higher average cumulative ROR while lower AUM funds managers taken medium to low risk and return comparativly lower average cumulative ROR.
  
  * Since segment size is very right skewed, this observations are not stongly significant. it would have been more significant if segment size distribution was approximately uniform.